In [1]:
from utility_common import *
from utility_plotter import *
#import ROOT
from IPython.display import clear_output
## add plotting option and load plotting parameter table
%matplotlib inline
#%qtconsole --style monokai
#plt.style.use('classic')

plt.rc("font",size=10)
plt.rc("savefig",dpi=300)
plt.rc("figure",facecolor="w",figsize=(6,4))


In [2]:
def makePlot(selection,nbjetcut):
    
    pickledir  =  "/mnt/data/zchen/Analysis/pickle/{}/".format(selection)
    if selection is "emu2":
        pickledir  =  "/mnt/data/zchen/Analysis/pickle/emu/"
        
    MCzz = LoadDataframe(pickledir + "mcdiboson")
    MCdy = LoadDataframe(pickledir + "mcdy")
    MCt  = LoadDataframe(pickledir + "mct")
    MCtt = LoadDataframe(pickledir + "mctt")
    
    MCTotay = pd.concat([MCzz,MCdy,MCt,MCtt],ignore_index=True)
    
    
    cuts = GetSelectionCut(selection,nbjetcut,shiftEnergyScale=None, shiftJet=None)
    MC   = MCTotay.query(cuts)
    

    if selection in ["e4j","mu4j"]:
        pp = pd.read_csv("plotparameters4j.csv")
    else:
        pp = pd.read_csv("plotparameters.csv")

    for index, row in pp.iterrows():
        v,a,b,step,xl = row["var"],row["lower"],row["upper"],row["step"],row["xlabel"]
        mybins = np.arange(a,b,step)
        mybinscenter = mybins[0:-1]+step/2

        fig, axes = plt.subplots(2, 1, sharex=True, facecolor="w",
                                 gridspec_kw={'height_ratios':[2,1]},
                                 figsize=(6,6))
        fig.subplots_adjust(hspace=0)

        ax   = axes[0] 
        cent = ax.hist(MC[v],weights=MC["eventWeight"],bins=mybins,histtype="step",color='k',alpha=0,lw=1,label=None)[0]

        for iname,name in enumerate(["BTag","Mistag","JES","JER"]):
            cutsUp = GetSelectionCut(selection,nbjetcut,shiftEnergyScale=None, shiftJet=name+"Up")
            cutsDown = GetSelectionCut(selection,nbjetcut,shiftEnergyScale=None, shiftJet=name+"Down")
            MCUp = MCTotay.query(cutsUp)
            MCDown = MCTotay.query(cutsDown)
            
            ax   = axes[0] 
            up0  = np.histogram(MCUp[v],  weights=MCUp["eventWeight"],bins=mybins)[0]
            down0= np.histogram(MCDown[v],weights=MCDown["eventWeight"],bins=mybins)[0]
            up   = (up0+down0)/2+np.abs(up0-down0)/2
            down = (up0+down0)/2-np.abs(up0-down0)/2
            ax.bar(mybins[:-1]+step/2, up-down, step,bottom=down,color="C{}".format(iname),label=name,alpha=0.6)
            ax.legend(fontsize=14)
            ax.grid(True,linestyle="--",alpha=0.5)

            ax = axes[1]
            nonzero = cent>0
            downratio = down[nonzero]/(cent[nonzero])
            upratio = up[nonzero]/cent[nonzero]
            ax.fill_between(mybinscenter[nonzero],upratio,downratio,color="C{}".format(iname),alpha=0.6)


        ax = axes[0]
        cent = ax.hist(MC[v],weights=MC["eventWeight"],bins=mybins,histtype="step",color='k',alpha=1,lw=1,label="nominal")[0]
        ax.legend(fontsize=14)
        

        ax = axes[1]
        ax.axhline(1,color="k")
        ax.set_ylim(0.95,1.05)
        ax.grid(True,linestyle="--",alpha=0.5)

        ax.set_xlabel(xl,fontsize=12)
        
        plt.savefig("../../plot/{}/jetSystematics/{}.png".format(selection,v),dpi=300)
        
        print("making plots nbjet{0}: {1}/{2}".format(nbjetcut,index,len(pp)))
        clear_output(wait=True)
        plt.close()


In [3]:
    #for selection in ["emu","mumu","mutau","ee","emu2","etau","mu4j","e4j"]:
    makePlot("e4j","==1")

making plots nbjet==1: 23/24
